# How to test:

model output on test set is saved; just use it.

In [1]:
from google.colab import drive
drive.mount('./drive')

Mounted at ./drive


In [19]:
import pandas as pd
import numpy as np
df = pd.read_csv('/content/drive/MyDrive/parsT5_QA/model_4/outs.csv', index_col=0)
df

,pred,target
0,بدون پاسخ,بدون پاسخ
1,بدون پاسخ,",اسفنج,اسفنج,اسفنج"
2,بدون پاسخ,بدون پاسخ
3,نوآوری در فن آوری,",جزئی,جزئی,جزئی"
4,ثبت نام اقامت,بدون پاسخ
...,...,...
6651,34 میلیون پوند,",34 میلیون پوند,34 میلیون پوند,34 میلیون پوند ..."
6652,نمونه های زمین، ماه ، مریخ و شهاب سنگ ها,بدون پاسخ
6653,بدون پاسخ,بدون پاسخ
6654,بدون پاسخ,بدون پاسخ


In [11]:
np.sum(df.pred == df.target)/6656

0.45102163461538464

In [20]:
count = 0
for i in range(len(df)):
    for j in df['target'][i].split(','):
        if j == df['pred'][i]:
            count += 1
            break
count/6656

0.4621394230769231

# How to predict:

download model files.

In [4]:
! gdown 1Lcs5eGTIhy0JUY9FW2pn-80m3CHyVtvQ

Downloading...
From: https://drive.google.com/uc?id=1Lcs5eGTIhy0JUY9FW2pn-80m3CHyVtvQ
To: /content/ParsT5.zip
100% 919M/919M [00:03<00:00, 234MB/s]


unzip them.

In [5]:
! unzip ParsT5.zip

Archive:  ParsT5.zip
   creating: content/drive/MyDrive/parsT5_QA/model_4/
  inflating: content/drive/MyDrive/parsT5_QA/model_4/config.json  
  inflating: content/drive/MyDrive/parsT5_QA/model_4/pytorch_model.bin  
  inflating: content/drive/MyDrive/parsT5_QA/model_4/tokenizer_config.json  
  inflating: content/drive/MyDrive/parsT5_QA/model_4/special_tokens_map.json  
  inflating: content/drive/MyDrive/parsT5_QA/model_4/tokenizer.json  
  inflating: content/drive/MyDrive/parsT5_QA/model_4/outs.csv  


In [ ]:
!pip install transformers

input should have following format:  
متن: -پاراگراف مورد نظر-، پرسش: -پرسش مورد نظر-

In [1]:
from transformers import T5ForConditionalGeneration, AutoTokenizer
context = 'خوب، بد، زشت یک فیلم درژانر وسترن اسپاگتی حماسی است که توسط سرجو لئونه در سال ۱۹۶۶ در ایتالیا ساخته شد. زبانی که بازیگران این فیلم به آن تکلم می‌کنند مخلوطی از ایتالیایی و انگلیسی است. این فیلم سومین (و آخرین) فیلم از سه‌گانهٔ دلار (Dollars Trilogy) سرجو لئونه است. این فیلم در حال حاضر در فهرست ۲۵۰ فیلم برتر تاریخ سینما در وب‌گاه IMDB با امتیاز ۸٫۸ از ۱۰، رتبهٔ هشتم را به خود اختصاص داده‌است و به عنوان بهترین فیلم وسترن تاریخ سینمای جهان شناخته می‌شود. «خوب» (کلینت ایستوود، در فیلم، با نام «بلوندی») و «زشت» (ایلای والاک، در فیلم، با نام «توکو») با هم کار می‌کنند و با شگرد خاصی، به گول زدن کلانترهای مناطق مختلف و پول درآوردن از این راه می‌پردازند. «بد» (لی وان کلیف) آدمکشی حرفه‌ای است که به‌خاطر پول حاضر به انجام هر کاری است. «بد»، که در فیلم او را «اِنجل آیز (اِینجل آیز)» (به انگلیسی: Angel Eyes) صدا می‌کنند. به‌دنبال گنجی است که در طی جنگ‌های داخلی آمریکا، به دست سربازی به نام «جکسون»، که بعدها به «کارسون» نامش را تغییر داده، مخفی شده‌است.'
question = 'در فیلم خوب بد زشت شخصیت ها کجایی صحبت می کنند؟'
input = 'متن: ' + context + '، پرسش: ' + question

now just load the tokenizer and the trained model. use tokenizer to tokenize input, use the model to generate output tokens, aaaand use the decoder again to generate output string. remove padding and stop tokens.

In [2]:
model_path = '/content/content/drive/MyDrive/parsT5_QA/model_4'
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = T5ForConditionalGeneration.from_pretrained(model_path)

input_ids = tokenizer.encode(input, return_tensors='pt')
output_ids = model.generate(input_ids, max_length=150, num_beams=2, repetition_penalty=2.5, length_penalty=1.0, early_stopping=True)
output = ' '.join([tokenizer.decode(id) for id in output_ids])
print(output.replace('<pad>', '').replace('</s>', '').strip())

بدون پاسخ


the answer was wrong though :(((  
  sorry that was the extent of my resources and abilities.